In [ ]:
import logging
import yaml

import os
import sqlite3
import pandas as pd

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
import asyncio
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

from astropy.utils import iers
iers.conf.auto_download = False

test_message = "Rotator Software Re-verification"

In [ ]:
d = salobj.Domain()
script = salobj.Controller("Script", index=1)
rot = salobj.Remote(d, "Rotator")
mtptg = salobj.Remote(d, "MTPtg")

await asyncio.gather(rot.start_task,
                     mtptg.start_task,
                     script.start_task)

In [ ]:
await asyncio.sleep(1.)
await salobj.set_summary_state(rot, salobj.State.ENABLED)
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

In [ ]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
def parallactic_angle(location, lst, target):
    """
    Calculate the parallactic angle.
    Parameters
    ----------
    time : `~astropy.time.Time`
        Observation time.
    target : `~astroplan.FixedTarget` or `~astropy.coordinates.SkyCoord` or list
        Target celestial object(s).
    grid_times_targets: bool
        If True, the target object will have extra dimensions packed onto the end,
        so that calculations with M targets and N times will return an (M, N)
        shaped result. Otherwise, we rely on broadcasting the shapes together
        using standard numpy rules.
    Returns
    -------
    `~astropy.coordinates.Angle`
        Parallactic angle.
    Notes
    -----
    The parallactic angle is the angle between the great circle that
    intersects a celestial object and the zenith, and the object's hour
    circle [1]_.
    .. [1] https://en.wikipedia.org/wiki/Parallactic_angle
    """
    # Eqn (14.1) of Meeus' Astronomical Algorithms
    H = (lst - target.ra).radian
    q = np.arctan2(np.sin(H),
                   (np.tan(location.lat.radian) *
                    np.cos(target.dec.radian) -
                    np.sin(target.dec.radian)*np.cos(H)))*u.rad
    return Angle(q)

Test Sequence #1 - PositionSet and Move Commands

In [ ]:
await rot.cmd_positionSet.set_start(angle=12.0)

In [ ]:
await rot.cmd_positionSet.set_start(angle=15.0)

In [ ]:
await rot.cmd_move.start()

Test Sequence #2 - Stop Command

In [ ]:
await rot.cmd_positionSet.set_start(angle=50.0)

In [ ]:
await rot.cmd_move.start()
await asyncio.sleep(5.)
await rot.cmd_stop.start()

In [ ]:
await rot.cmd_positionSet.set_start(angle=0.0)
await asyncio.sleep(0.1)
await rot.cmd_move.start()

Test startTrack without track

In [ ]:
await rot.cmd_positionSet.set_start(angle=0.0)
await asyncio.sleep(0.1)
await rot.cmd_move.start()

In [ ]:
await rot.cmd_trackStart.start()

In [ ]:
await rot.cmd_trackStart.start()
await rot.cmd_track.set_start(angle=0.0,velocity=0.068,tai=salobj.current_tai()+0.05)
await asyncio.sleep(0.1)
await rot.cmd_stop.start()
await asyncio.sleep(100.)
await rot.cmd_trackStart.start()

In [ ]:
await rot.cmd_stop.start()

Test Sequence #4 - Track and TrackStart Commands

In [ ]:
tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

vel = 0.068
dt = (95 + -85) / vel
dpos = vel * 0.1
steps = int(dt / 0.1)

try:
    await rot.cmd_trackStart.start(timeout=30.)
except salobj.AckError:
    pass

for i in range(steps):
    if i ==0:
        print("Max steps is %d." %steps)
    pos = -85 - i*dpos
    await rot.cmd_track.set_start(
        angle=pos,
        velocity=vel,
        tai=salobj.current_tai(),
        timeout=10.
    )
    print(i)
    print(pos)
    await asyncio.sleep(0.05)

#await rot.cmd_stop.start(timeout=30.)


In [ ]:
print(pos)

In [ ]:
print("Azimuth = 0 and rot_tel= 60 deg")

alt = 45. * u.deg
az = 0. * u.deg
rot_tel = Angle(10, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 30s")
await asyncio.sleep(30.)

await mtptg.cmd_stopTracking.start(timeout=10.)

await asyncio.sleep(0.1)

await rot.cmd_stop.start(timeout=10.)

Test Sequence #5 - Track and TrackStart Commands

In [ ]:
print("Azimuth = 180 and rot_tel= -60 deg")

alt = 45. * u.deg
az = 180. * u.deg
rot_tel = Angle(-60, unit= u.deg) 

target_name="Rotator test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.tai, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

# Compute RA/Dec for commanded az/el
cmd_elaz = AltAz(alt=alt, az=az, 
                 obstime=curr_time_mtptg.tai, 
                 location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
lst = Angle(time_data.lst, unit=u.hour)

para_ang=parallactic_angle(location, lst, cmd_radec).to(u.deg)
print(f"Alt: {alt}\nAz: {az}\nRotTel: {rot_tel}\nParaAng: {para_ang}")


# Calculating the other parameters     
rot_pa = para_ang-180*u.deg+rot_tel
print(rot_pa)

await rot.cmd_trackStart.start(timeout=30.)
    
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    targetInstance=ATPtg.TargetInstances.CURRENT,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg,
    rotFrame=ATPtg.RotFrame.TARGET,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
    
print("Waiting 30s")
await asyncio.sleep(30.)

await mtptg.cmd_stopTracking.start(timeout=10.)

await asyncio.sleep(0.1)

await rot.cmd_stop.start(timeout=10.)

Test Sequence #6 - configureVelocity

In [ ]:
await rot.cmd_configureVelocity.set_start(vlimit=4.)

In [ ]:
await rot.cmd_configureVelocity.set_start(vlimit=0.5)

In [ ]:
tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

newPos = cpos + 10.

await rot.cmd_positionSet.set_start(angle=newPos)
await rot.cmd_move.start(timeout=10.)

Test Sequence #7 - configureAcceleration

In [ ]:
await rot.cmd_configureAcceleration.set_start(alimit=2.)

In [ ]:
await rot.cmd_configureAcceleration.set_start(alimit=0.05)

In [ ]:
tel = rot.tel_Application.get()
cpos = tel.Position
print(cpos)

newPos = cpos + 10.

await rot.cmd_positionSet.set_start(angle=newPos)
await rot.cmd_move.start(timeout=10.)

Section 3.3.2 State Machine

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await rot.cmd_start.start(timeout=10.) #probably won't run these

In [ ]:
await rot.cmd_enable.start(timeout=10.)

In [ ]:
await rot.cmd_disable.start(timeout=10.)

In [ ]:
await rot.cmd_standby.start(timeout=10.)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

Additional Error Handling Commands

In [ ]:
await rot.cmd_clearError.start(timeout=10)

In [ ]:
await rot.cmd_exitControl.start(timeout=10.)

In [ ]:
await rot.cmd_enterControl.start(timeout=10.)

In [ ]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

In [ ]:
await rot.cmd_stop.start(timeout=10.)

In [ ]:
await rot.cmd_configureVelocity.set_start(vlimit=3.5)

In [ ]:
await rot.cmd_configureAcceleration.set_start(alimit=1.0)